In [1]:
from robot_descriptions.skydio_x2_description import URDF_PATH
from darli.backend import PinocchioBackend, CasadiBackend
from darli.modeling import Robot, Functional
from darli.modeling.integrators import ForwardEuler, RK4
import numpy as np

model = Functional(CasadiBackend(URDF_PATH))
selector = np.array(
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0],
        [1.0, 1.0, 1.0, 1.0],
        [-0.18, 0.18, 0.18, -0.18],
        [0.14, 0.14, -0.14, -0.14],
        [-0.0201, 0.0201, 0.0201, -0.0201],
    ]
)
model.update_selector(selector)

In [2]:
np.random.seed(0)
tau = np.random.randn(model.nu)

tau

array([1.76405235, 0.40015721, 0.97873798, 2.2408932 ])

In [3]:
q0 = np.array([0, 0, 0, 0, 0, 0, 1])
v0 = np.array([0, 0, 0, 0, 0, 0])

x0 = np.concatenate([q0, v0])


## Functional way

In [4]:
euler_roll = model.state_space.rollout(dt=2e-3, n_steps=100, integrator=ForwardEuler)

%timeit euler_roll

52.7 ns ± 3.43 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [5]:
rk4_roll = model.state_space.rollout(dt=2e-3, n_steps=100, integrator=RK4)

%timeit rk4_roll

49.1 ns ± 2.72 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


## Pinocchio way

In [6]:
model = Robot(PinocchioBackend(URDF_PATH))
selector = np.array(
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0],
        [1.0, 1.0, 1.0, 1.0],
        [-0.18, 0.18, 0.18, -0.18],
        [0.14, 0.14, -0.14, -0.14],
        [-0.0201, 0.0201, 0.0201, -0.0201],
    ]
)
model.update_selector(selector)

In [7]:
euler_state = model.state_space.rollout(x0, tau, 2e-3, 100, ForwardEuler)

euler_state, "quaternion norm", np.linalg.norm(euler_state[3:7])

(array([ 0.00457161, -0.01065659, -0.11412443, -0.13071001, -0.05838671,
        -0.0124036 ,  0.98962217, -0.1032257 ,  0.21819858, -1.08951141,
        -2.59886166, -1.16024245, -0.22845503]),
 'quaternion norm',
 0.9999999999999999)

In [8]:
rk4_state = model.state_space.rollout(x0, tau, 2e-3, 100, RK4)

rk4_state, "quaternion norm", np.linalg.norm(rk4_state[3:7])

(array([ 1.23542936e-04, -2.75823880e-04, -2.29782590e-03, -2.59450405e-03,
        -1.16291972e-03, -2.64425683e-04,  9.99995923e-01,  1.20941934e-04,
        -2.69829401e-04, -2.29866843e-03, -5.18901516e-03, -2.32584260e-03,
        -5.28852084e-04]),
 'quaternion norm',
 1.0)